# Глубинное обучение 1 / Введение в глубинное обучение, ФКН ВШЭ

## Домашнее задание 2: сверточный классификатор.

### Общая информация

Оценка после штрафа после мягкого дедлайна вычисляется по формуле $M_{\text{penalty}} = M_{\text{full}} \cdot 0.85^{t/1440}$, где $M_{\text{full}}$ — полная оценка за работу без учета штрафа, а $t$ — время в минутах, прошедшее после мягкого дедлайна (округление до двух цифр после запятой). Таким образом, спустя первые сутки после мягкого дедлайна вы не можете получить оценку выше 8.5, а если сдать через четыре дня после мягкого дедлайна, то ваш максимум — 5.22 балла.

### Оценивание и штрафы

Максимально допустимая оценка за работу — 10 баллов. Сдавать задание после указанного срока сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

### О задании

В этом задании вам предстоит познакомиться со сверточными сетями и их обучением для классификации изображений с использованием библиотеки PyTorch.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split

## 0. Загрузка данных

Работать мы будем с набором данных [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html). CIFAR10 представляет собой набор изображений 32х32 пикселя, разделенных на 10 классов.

![title](https://pytorch.org/tutorials/_images/cifar10.png)




Набор данных уже определен в <code>torchvision.datasets</code>, так что возьмем его оттуда. 



In [2]:
def get_cifar10_data(batch_size, transform_train):
    torch.manual_seed(0)
    np.random.seed(0)

    transform_test = transforms.Compose(
        [transforms.ToTensor(),
         # Переводим цвета пикселей в отрезок [-1, 1]
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    
    # Загружаем данные
    trainvalset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                               download=True, transform=transform_train)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transform_test)
    
    # В датасете определено разбиение только на train и test,
    # так что валидацию дополнительно выделяем из обучающей выборки
    train_idx, valid_idx = train_test_split(np.arange(len(trainvalset)), test_size=0.3, 
                                            shuffle=True, random_state=0)
    trainset = torch.utils.data.Subset(trainvalset, train_idx)
    valset = torch.utils.data.Subset(trainvalset, valid_idx)

    train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                               shuffle=True, num_workers=2)
    val_loader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                             shuffle=False, num_workers=2)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                              shuffle=False, num_workers=2)
    
    return train_loader, val_loader, test_loader
    

In [3]:
transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

train_loader, val_loader, test_loader = get_cifar10_data(batch_size=64, 
                                                         transform_train=transform)

Files already downloaded and verified
Files already downloaded and verified


Посмотрим на изображения:

In [4]:
def imshow(img):
    img = img / 2 + 0.5    
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


dataiter = iter(train_loader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images[:4]))

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
print(*[classes[labels[i]] for i in range(4)])

AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute 'next'

## 1. Задание сверточной сети (3 балла)

Теперь нам нужно задать сверточную нейронную сеть, которую мы будем обучать классифицировать изображения.

Используем сеть, основанную на одном блоке архитектуры ResNet.

<img src="https://i.ibb.co/2hg962h/basic-block.png" width="500"/>

__Указания:__

- Все сверточные слои должны иметь 32 выходных канала, а также не должны изменять ширину и высоту изображения.
- Выход блока сократите до размерности 32х4х4, применив average pooling.
- Для получения итоговых логитов, распрямите выход пулинга в вектор из 512 элементов, а затем пропустите его через линейный слой.

**Задание 1.1 (3 балла).**

Определите архитектуру сети соответственно схеме и указаниям выше.

Ключевые слова: <code>Conv2d</code>, <code>BatchNorm2d</code>, <code>AvgPool2d</code>.

In [6]:
n_classes = 10

class BasicBlockNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        out_channels = 32  

        self.block = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=out_channels, kernel_size=3, padding='same'),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, padding='same'),
            nn.BatchNorm2d(out_channels),
        )
        
        self.shortcut = nn.Conv2d(in_channels=3, out_channels=out_channels, kernel_size=1, padding='same')
        self.relu = nn.ReLU()
        self.pool = nn.AvgPool2d(kernel_size=8)
        self.flatten = nn.Flatten(start_dim=1)
        self.linear = nn.Linear(in_features=512, out_features=n_classes)
        
    def forward(self, x):
        residual = self.shortcut(x)
        out = self.block(x)
        out += residual 
        out = self.relu(out) 
        out = self.pool(out) 
        out = self.flatten(out) 
        out = self.linear(out)  
        return out

In [7]:
net = BasicBlockNet()
net

BasicBlockNet(
  (block): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (shortcut): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)
  (relu): ReLU()
  (pool): AvgPool2d(kernel_size=8, stride=8, padding=0)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=512, out_features=10, bias=True)
)

Проверим, что выход сети имеет корректную размерность:

In [8]:
assert net(torch.zeros((10, 3, 32, 32))).shape == (10, 10)

Чтобы проводить вычисления на GPU, в PyTorch необходимо руками перекладывать объекты, с которыми вы хотите проводить вычисления, на графический ускоритель. Это делается следующим образрм:

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [10]:
net = net.to(device)

Подключение GPU в google.colab:

**Среда выполнения** -> **Сменить среду выполнения** -> **Аппаратный ускоритель** -> **GPU**

## 2. Обучение и тестирование модели (3 балла)

**Задание 2.1 (2 балла).** Переходим к обучению модели. Заполните пропуски в функциях <code>test</code> и <code>train_epoch</code>. В качестве функции потерь будем использовать [кросс-энтропию](https://pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html), а в качестве метрики качества accuracy.

In [12]:
def test(model, loader):
    loss_log = []
    acc_log = []
    model.eval()
    
    for data, target in loader:
        
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            logits = model(data)  
            loss = nn.CrossEntropyLoss()(logits, target) 

        acc = torch.sum((logits.argmax(dim=1) == target).to(torch.float32)) / len(target)
        loss_log.append(loss.item())
        acc_log.append(acc.item())
        
    return np.mean(loss_log), np.mean(acc_log)

def train_epoch(model, optimizer, train_loader):
    loss_log = []
    acc_log = []
    model.train()
    
    for data, target in train_loader:
        
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        logits = model(data)
        loss = nn.CrossEntropyLoss()(logits, target)
        loss.backward()
        optimizer.step()

        
        acc = torch.sum((logits.argmax(dim=1) == target).to(torch.float32)) / len(target)
        loss_log.append(loss.item())  
        acc_log.append(acc.item())

    return loss_log, acc_log

def train(model, optimizer, n_epochs, train_loader, val_loader, scheduler=None):
    train_loss_log, train_acc_log, val_loss_log, val_acc_log = [], [], [], []

    for epoch in range(n_epochs):
        train_loss, train_acc = train_epoch(model, optimizer, train_loader)
        val_loss, val_acc = test(model, val_loader)
        
        train_loss_log.extend(train_loss)
        train_acc_log.extend(train_acc)
        
        val_loss_log.append(val_loss)
        val_acc_log.append(val_acc)

        print(f"Epoch {epoch}")
        print(f" train loss: {np.mean(train_loss)}, train acc: {np.mean(train_acc)}")
        print(f" val loss: {val_loss}, val acc: {val_acc}\n")
        
        if scheduler is not None:
            scheduler.step()

    return train_loss_log, train_acc_log, val_loss_log, val_acc_log

Запустим обучение модели. В качестве оптимизатора будем использовать стохастический градиентный спуск, который является де-факто стандартом в задачах компьютерного зрения (наравне с <code>Adam</code>).

__Замечание:__ Для достижения наилучшего качества в нашем случае потребуется обучать модель несколько сотен эпох. Однако в целях экономии вашего времени и сил, во всех экспериментах мы ограничимся 20 эпохами.

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
train_loss_log, train_acc_log, val_loss_log, val_acc_log = train(net, optimizer, 20, train_loader, val_loader)

Посчитайте точность на тестовой выборке:

In [ ]:
loss, accuracy = test(net, test_loader)
print(f'Loss = {loss}, Точность модели = {accuracy}')

Если вы все сделали правильно, у вас должна была получиться точность $\ge 67\%$.

**Задание 2.2 (1 балл).** Постройте графики зависимости функции потерь и точности классификации от номера шага оптимизатора. На каждом графике расположите данные и для обучающей и для валидационной выборки, итого у вас должно получиться два графика. Обратите внимание, что на обучающей выборке эти данные считаются по каждому батчу, на валидационной же они считаются по всей выборке раз в эпоху.

In [ ]:
plt.figure(figsize=(12, 6))

plt.plot(train_loss_log, label='Обучающая выборка', alpha=0.7)

val_loss_steps = [val_loss_log[i] for i in range(len(val_loss_log))]
plt.plot(
    np.linspace(0, len(train_loss_log), len(val_loss_steps)), 
    val_loss_steps, 
    label='Валидационная выборка', 
    marker='o'
)

plt.title('Зависимость функции потерь от номера шага оптимизатора')
plt.xlabel('Шаг оптимизатора')
plt.ylabel('Функция потерь')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(12, 6))

plt.plot(train_acc_log, label='Обучающая выборка', alpha=0.7)

val_acc_steps = [val_acc_log[i] for i in range(len(val_acc_log))]
plt.plot(
    np.linspace(0, len(train_acc_log), len(val_acc_steps)), 
    val_acc_steps, 
    label='Валидационная выборка', 
    marker='o'
)

plt.title('Зависимость точности классификации от номера шага оптимизатора')
plt.xlabel('Шаг оптимизатора')
plt.ylabel('Точность')
plt.legend()
plt.grid(True)
plt.show()


## 3. Расписание длины шага (2 балла)

С курса "Машинное обучение 1" вы уже должны знать, что сходимость стохастического градиентного спуска мы можем теоретически гарантировать только если будем определенным образом со временем уменьшать длину шага. На практике при обучении нейронных сетей такая техника оказывается очень полезной, однако теоретически обоснованными способами уменьшения длины шага фантазия не ограничивается.

Одним из простейших способов является кусочно постоянная функция: на нескольких фиксированных эпохах уменьшаем длину шага в константу раз.

In [ ]:
net = BasicBlockNet().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 15], gamma=0.1)
tr_loss_log, tr_acc_log, val_loss_log, val_acc_log = train(net, optimizer, 20, train_loader, val_loader, scheduler)

Посчитайте точность на тестовой выборке:

In [ ]:
loss, accuracy = test(net, test_loader)
print(f'Loss = {loss}, Точность модели = {accuracy}')

**Задание 3.0 (0.5 баллов).** Здесь может возникнуть вопрос: а что будет, если мы не будем уменьшать длину шага в процессе обучения, а сразу возьмем констатную, равную значению нашей кусочно-постоянной функции на последних эпохах, то есть $0.001$ в нашем случае. Запустите обучение и проверьте, что в таком случае мы получим худшее качество на тестовой выборке.

In [ ]:
net = BasicBlockNet().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
train_loss_log, train_acc_log, val_loss_log, val_acc_log = train(net, optimizer, 20, train_loader, val_loader)

In [ ]:
loss, accuracy = test(net, test_loader)
print(f'Loss = {loss}, Точность модели = {accuracy}')

Качество действительно ухудшилось

**Задание 3.1 (1.5 балла).** Изучите, какие еще способы уменьшения длины шага представлены в <code>torch.optim.lr_scheduler</code>. Выберите несколько из них, объясните, как они устроены, и обучите модель с ними. Удалось ли добиться улучшения качества на тестовой выборке?

1. ExponentialLR -  умножает lr на гиперпараметр gamma после каждой эпохи

In [ ]:
net = BasicBlockNet().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)
tr_loss_log, tr_acc_log, val_loss_log, val_acc_log = train(net, optimizer, 20, train_loader, val_loader, scheduler)

In [ ]:
loss, accuracy = test(net, test_loader)
print(f'Loss = {loss}, Точность модели = {accuracy}')

2. CosineAnnealingLR - косинусное снижение (cosine annealing schedule).

Значение $\eta_{\text{max}}$ устанавливается равным начальному значению скорости обучения, а $T_{\text{cur}}$ — это количество эпох с момента последнего перезапуска:

$
\eta_t = \eta_{\text{min}} + \frac{1}{2} (\eta_{\text{max}} - \eta_{\text{min}}) \left( 1 + \cos\left(\frac{T_{\text{cur}}}{T_{\text{max}}} \pi \right) \right), \quad T_{\text{cur}} \neq (2k + 1) T_{\text{max}}
$

$
\eta_{t+1} = \eta_t + \frac{1}{2} (\eta_{\text{max}} - \eta_{\text{min}}) \left( 1 - \cos\left(\frac{1}{T_{\text{max}}} \pi \right) \right), \quad T_{\text{cur}} = (2k + 1) T_{\text{max}}
$


In [ ]:
net = BasicBlockNet().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader)*20)
tr_loss_log, tr_acc_log, val_loss_log, val_acc_log = train(net, optimizer, 20, train_loader, val_loader, scheduler)

In [ ]:
loss, accuracy = test(net, test_loader)
print(f'Loss = {loss}, Точность модели = {accuracy}')

3. StepLR - снижает значение скорости обучения для каждой группы параметров в заданное число раз (gamma) через каждые step_size эпох.

In [ ]:
net = BasicBlockNet().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
tr_loss_log, tr_acc_log, val_loss_log, val_acc_log = train(net, optimizer, 20, train_loader, val_loader, scheduler)

In [ ]:
loss, accuracy = test(net, test_loader)
print(f'Loss = {loss}, Точность модели = {accuracy}')

4. OneCycleLR - сначала скорость обучения увеличивается от начального значения до максимального, а затем снижается с максимального значения до минимального, которое значительно ниже начального. OneCycleLR изменяет скорость обучения после каждой батча 

In [ ]:
net = BasicBlockNet().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.3, steps_per_epoch=len(train_loader), epochs=20)
tr_loss_log, tr_acc_log, val_loss_log, val_acc_log = train(net, optimizer, 20, train_loader, val_loader, scheduler)

In [ ]:
loss, accuracy = test(net, test_loader)
print(f'Loss = {loss}, Точность модели = {accuracy}')

5. CyclicLR - этот способ изменяет скорость обучения в пределах двух границ с постоянной частотой, как описано в статье Cyclical Learning Rates for Training Neural Networks. Расстояние между этими границами может изменяться на основе итераций или циклов.

In [ ]:
net = BasicBlockNet().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.1, step_size_up=5, mode="triangular2")
tr_loss_log, tr_acc_log, val_loss_log, val_acc_log = train(net, optimizer, 20, train_loader, val_loader, scheduler)

In [ ]:
loss, accuracy = test(net, test_loader)
print(f'Loss = {loss}, Точность модели = {accuracy}')

Видим, что точность модели немного улучшилась при использовании CyclicLR
и ухудшилась при всех остальных способах 

## 4. Аугментации данных (2 балла)

Еще одной стандартной техникой, применяющейся в глубинном обучении, а особенно часто в компьютерном зрении, являются аугментации данных. Суть аугментаций состоит в том, что мы можем некоторым синтетическим образом видоизменять объекты обучающей выборки, тем самым расширяя ее, а также делая итоговую модель более устойчивой к таким изменениям.

Простейшая аугментация, которую можно применить к картинкам — разворот картинки по горизонтальной оси. То есть при обучении модели с вероятностью $0.5$ мы будем разворачивать картинку из обучающей выборки.

In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images[:4]))

imshow(torchvision.utils.make_grid(transforms.functional.hflip(images[:4])))

Наиболее удобным способом работы с аугментациями в PyTorch является их задание в списке <code>transforms</code>, который затем передается в загрузчик данных. Обучим нашу сеть, применяя горизонтальные повороты:

In [17]:
transform = transforms.Compose(
        [transforms.RandomHorizontalFlip(),
         transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

train_loader, val_loader, test_loader = get_cifar10_data(batch_size=64, transform_train=transform)

net = BasicBlockNet().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 15], gamma=0.1)
tr_loss_log, tr_acc_log, val_loss_log, val_acc_log = train(net, optimizer, 20, train_loader, val_loader, scheduler)

Files already downloaded and verified
Files already downloaded and verified
Epoch 0
 train loss: 1.4819418806480533, train acc: 0.4699334029305788
 val loss: 1.2903767192617375, val acc: 0.537987588694755

Epoch 1
 train loss: 1.1751501410927154, train acc: 0.5877799360146252
 val loss: 1.0969221188666973, val acc: 0.6188386525245423

Epoch 2
 train loss: 1.050908883921189, train acc: 0.6341366871401423
 val loss: 1.049557152707526, val acc: 0.6383865247381494

Epoch 3
 train loss: 0.99197962024748, train acc: 0.6563397753827097
 val loss: 0.9752251924352443, val acc: 0.6635195034615538

Epoch 4
 train loss: 0.9486573364660552, train acc: 0.6700509271194556
 val loss: 0.9910797418432032, val acc: 0.6572251773895101

Epoch 5
 train loss: 0.9166564265814299, train acc: 0.683027552839171
 val loss: 0.9286246951590194, val acc: 0.6742907800572984

Epoch 6
 train loss: 0.8988320501129632, train acc: 0.6927314572839894
 val loss: 0.9507466516596206, val acc: 0.6666666667512122

Epoch 7
 trai

Посчитайте точность на тестовой выборке:

In [18]:
loss, accuracy = test(net, test_loader)
print(f'Loss = {loss}, Точность модели = {accuracy}')

Loss = 0.78858767971871, Точность модели = 0.7298964968152867


**Задание 4.1 (2 балла).** Изучите, какие еще способы аугментаций изображений представлены в <code>torchvision.transforms</code>. Выберите несколько из них, объясните, как они устроены, и обучите модель с ними (по отдельности и вместе). Удалось ли добиться улучшения качества на тестовой выборке?

1. RandomRotation случайно вращает изображение на указанный угол

In [19]:
transform = transforms.Compose([
    transforms.RandomRotation(degrees=(-30, 30)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_loader, val_loader, test_loader = get_cifar10_data(batch_size=64, transform_train=transform)

net = BasicBlockNet().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 15], gamma=0.1)
tr_loss_log, tr_acc_log, val_loss_log, val_acc_log = train(net, optimizer, 20, train_loader, val_loader, scheduler)

Files already downloaded and verified
Files already downloaded and verified
Epoch 0
 train loss: 1.6132791698305873, train acc: 0.41962245362768225
 val loss: 1.4277350902557373, val acc: 0.48189273045418113

Epoch 1
 train loss: 1.392240122859613, train acc: 0.5045377383284421
 val loss: 1.281417847440598, val acc: 0.5492464538584364

Epoch 2
 train loss: 1.2953308241023007, train acc: 0.5433574366831039
 val loss: 1.2500418627515753, val acc: 0.5601728723404256

Epoch 3
 train loss: 1.2403516978624751, train acc: 0.5628958279079668
 val loss: 1.1949552769356586, val acc: 0.5807845744680851

Epoch 4
 train loss: 1.1915322946456057, train acc: 0.5799327500341579
 val loss: 1.2050676074433835, val acc: 0.5827349289934686

Epoch 5
 train loss: 1.1589220026729312, train acc: 0.5893999739146538
 val loss: 1.2276497366580557, val acc: 0.5713874114320633

Epoch 6
 train loss: 1.1312244284959336, train acc: 0.5988957625856347
 val loss: 1.1061254653524846, val acc: 0.6072916667512123

Epoch 7

In [20]:
loss, accuracy = test(net, test_loader)
print(f'Loss = {loss}, Точность модели = {accuracy}')

Loss = 0.8842636604977262, Точность модели = 0.6964570063694268


Качество модели ухудшилось

2. RandomGrayscale случайно преобразует изображение в градации серого с заданной вероятностью p

In [ ]:
transform = transforms.Compose([
    transforms.RandomGrayscale(p=0.3),  
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_loader, val_loader, test_loader = get_cifar10_data(batch_size=64, transform_train=transform)
net = BasicBlockNet().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 15], gamma=0.1)
tr_loss_log, tr_acc_log, val_loss_log, val_acc_log = train(net, optimizer, 20, train_loader, val_loader, scheduler)

Files already downloaded and verified
Files already downloaded and verified
Epoch 0
 train loss: 1.525947941504623, train acc: 0.45753215588424934
 val loss: 1.2651645954619062, val acc: 0.5523492908224147

Epoch 1
 train loss: 1.2006732561034739, train acc: 0.5842582920133741
 val loss: 1.156546888452895, val acc: 0.5993351063829787

Epoch 2
 train loss: 1.0896178261671468, train acc: 0.6218782645910705
 val loss: 1.128366657774499, val acc: 0.6136746454746165

Epoch 3
 train loss: 1.0378861389168874, train acc: 0.6410330699610318
 val loss: 1.02205875579347, val acc: 0.6429964539852548

Epoch 4
 train loss: 1.0029416377409284, train acc: 0.6524794332723949
 val loss: 1.0548620497926753, val acc: 0.635483155986096

Epoch 5
 train loss: 0.9690946990000918, train acc: 0.6649010838494658
 val loss: 1.003455032439942, val acc: 0.6530806736743197

Epoch 6
 train loss: 0.9451830922140718, train acc: 0.6722341016398051
 val loss: 1.0187926343146791, val acc: 0.6541666667512123

Epoch 7
 trai

In [ ]:
loss, accuracy = test(net, test_loader)
print(f'Loss = {loss}, Точность модели = {accuracy}')

Loss = 0.8141631372035689, Точность модели = 0.7224323248407644


Качество модели совсем немного ухудшилось

3. RandomSolarize инвертирует значения пикселей, которые превышают заданный порог threshold.

In [ ]:
transform = transforms.Compose([
    transforms.RandomSolarize(threshold=2.0), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_loader, val_loader, test_loader = get_cifar10_data(batch_size=64, transform_train=transform)

net = BasicBlockNet().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 15], gamma=0.1)
tr_loss_log, tr_acc_log, val_loss_log, val_acc_log = train(net, optimizer, 20, train_loader, val_loader, scheduler)

Files already downloaded and verified
Files already downloaded and verified
Epoch 0
 train loss: 1.738553910194411, train acc: 0.38314099634369286
 val loss: 1.4613562421595796, val acc: 0.4837544327086591

Epoch 1
 train loss: 1.3980688777440644, train acc: 0.5154413685502256
 val loss: 1.4241647101463155, val acc: 0.5209441489361702

Epoch 2
 train loss: 1.2823135075882977, train acc: 0.5564360799475605
 val loss: 1.2749588098931819, val acc: 0.561968085106383

Epoch 3
 train loss: 1.2094320018505047, train acc: 0.5834380712979891
 val loss: 1.3209133772139854, val acc: 0.5559397163543295

Epoch 4
 train loss: 1.1683819548957308, train acc: 0.5990793940898923
 val loss: 1.2850049883761305, val acc: 0.56657801422667

Epoch 5
 train loss: 1.1309369267449736, train acc: 0.6104441433327725
 val loss: 1.1917575143753214, val acc: 0.5918661348363187

Epoch 6
 train loss: 1.1050233191502377, train acc: 0.6195359428360432
 val loss: 1.1293186033025702, val acc: 0.6131648936170213

Epoch 7
 t

In [ ]:
loss, accuracy = test(net, test_loader)
print(f'Loss = {loss}, Точность модели = {accuracy}')

Loss = 0.9565982040326306, Точность модели = 0.6714769108280255


Качество модели ухудшилось сильнее

4. RandomPerspective выполняет аугментацию изображенияс помощью перспективного преобразования. Это означает, что изображение выглядит как будто оно было снято с определенного угла, добавляя эффект "перспективы"

In [ ]:
transform = transforms.Compose([
    transforms.RandomPerspective(distortion_scale=0.4, p=0.7),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_loader, val_loader, test_loader = get_cifar10_data(batch_size=64, transform_train=transform)

net = BasicBlockNet().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 15], gamma=0.1)
tr_loss_log, tr_acc_log, val_loss_log, val_acc_log = train(net, optimizer, 20, train_loader, val_loader, scheduler)

Files already downloaded and verified
Files already downloaded and verified
Epoch 0
 train loss: 1.681427236666845, train acc: 0.3933549882942841
 val loss: 1.4896121552649966, val acc: 0.4596409574468085

Epoch 1
 train loss: 1.418673541035905, train acc: 0.4900716570984075
 val loss: 1.3014566862836796, val acc: 0.5362810283265216

Epoch 2
 train loss: 1.3007483518319767, train acc: 0.5379953969549217
 val loss: 1.229397526446809, val acc: 0.5677969859001485

Epoch 3
 train loss: 1.216878668808632, train acc: 0.5691352180414705
 val loss: 1.1816012035024928, val acc: 0.5838209220703612

Epoch 4
 train loss: 1.1599473225351224, train acc: 0.5896162509482347
 val loss: 1.3226359095979243, val acc: 0.5502881206096487

Epoch 5
 train loss: 1.1307298814574804, train acc: 0.6029683011541419
 val loss: 1.1143428021288933, val acc: 0.611436170212766

Epoch 6
 train loss: 1.1035314754768526, train acc: 0.6112113149336095
 val loss: 1.1153416613315015, val acc: 0.6134530140998515

Epoch 7
 tra

In [ ]:
loss, accuracy = test(net, test_loader)
print(f'Loss = {loss}, Точность модели = {accuracy}')

Loss = 0.8882117123360846, Точность модели = 0.6978503184713376


Качество модели ухудшилось по сравнению с изначальным

Теперь применим вместе RandomRotation, RandomSolarize и RandomGrayscale

In [ ]:
transform = transforms.Compose([
         transforms.RandomSolarize(threshold=128, p=0.5),
         transforms.RandomGrayscale(p=0.3),
         transforms.RandomRotation(degrees=(-30, 30)),
         transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
train_loader, val_loader, test_loader = get_cifar10_data(batch_size=64, transform_train=transform)

net = BasicBlockNet().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 15], gamma=0.1)
tr_loss_log, tr_acc_log, val_loss_log, val_acc_log = train(net, optimizer, 20, train_loader, val_loader, scheduler)

Files already downloaded and verified
Files already downloaded and verified
Epoch 0
 train loss: 1.8586260433388802, train acc: 0.32732926349971153
 val loss: 1.7151867612879328, val acc: 0.3835992908224146

Epoch 1
 train loss: 1.647178137324192, train acc: 0.4160640833037863
 val loss: 1.5951880033980024, val acc: 0.42936613483631864

Epoch 2
 train loss: 1.55062673950544, train acc: 0.4499502154765225
 val loss: 1.5163004920837726, val acc: 0.46513741143206333

Epoch 3
 train loss: 1.4997347523568971, train acc: 0.47052510445035134
 val loss: 1.5685312124008828, val acc: 0.46327570930440376

Epoch 4
 train loss: 1.455651570935572, train acc: 0.48499526636474094
 val loss: 1.4524673319877461, val acc: 0.4876994680851064

Epoch 5
 train loss: 1.427612861723961, train acc: 0.4957968790962666
 val loss: 1.4403137952723402, val acc: 0.48854166662439386

Epoch 6
 train loss: 1.400738633093058, train acc: 0.5053457168817956
 val loss: 1.3829903044599168, val acc: 0.5072030142266699

Epoch 

In [ ]:
loss, accuracy = test(net, test_loader)
print(f'Loss = {loss}, Точность модели = {accuracy}')

Loss = 0.9645237106426506, Точность модели = 0.6667993630573248


Качество ухудшилось сильнее, чем при использовании каждого метода по отдельности

Похоже, что качество модели рассмотренными аугментациями улучшить не удалось, возможно из-за того, что при их применении модель становится сложнее и 20 эпох не достаточно 

## Бонус. Логирование в wandb (2 балла)

На практике специалиста по глубинному обучению часто встречаются ситуации, когда нейросеть учится на каком-то удаленном сервере. И обычно вам хочется отслеживать прогресс обучения, особенно когда время обучения модели исчисляется днями или неделями. Для таких целей существует несколько инструментов. Вероятно, самый популярный из них — [wandb](https://wandb.ai/site).

Ваша задача состоит в том, чтобы разобраться как им пользоваться, и повторить задания 2.1 и 2.2 с его использованием. Обучение вы можете запускать в этом же ноутбуке, но теперь вам необходимо через wandb логировать значения функции потерь и точности на обучающей выборке и на валидационной. Результатом работы должны быть ваш код и публичная ссылка на страничку с графиками, идентичными графикам в задании 2.2.

In [ ]:
%pip install wandb 

In [ ]:
wandb.login()

In [ ]:
import wandb
import random